# Part 11: Cerise and Denim FTW
We've now got just a bit of an idea about our perceptrons and how they are connected together. And if you think about it, 48 colors times 4 perceptrons in the middle layer, being 192 perceptrons, is not even close to the neurons a jellyfish has, not to mention a mouse or an elephant.

Oh well, let's see where we can get to next. Let's graduate from toddler to a 5-year-old level: Lots more colors to learn. We'll keep all the extra code from lesson 9, perturbing the data just as much as before, using 4 perceptrons per color label, splitting into training and test data. And we'll increase the batch size since we're about to drive up the number of examples into the hundreds of thousands.

In [1]:
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential
import keras.optimizers, keras.utils, numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

def train(rgbValues, colorNames, epochs = 3, perceptronsPerColorName = 4, batchSize = 16):
    """
    Trains a neural network to understand how to map color names to RGB triples.
    The provided lists of RGB triples must be floating point triples with each
    value in the range [0.0, 1.0], and the number of color names must be the same length.
    Different names are allowed to map to the same RGB triple.
    Returns a trained model that can be used for recognize().
    """

    # Convert the Python map RGB values into a numpy array needed for training.
    rgbNumpyArray = numpy.array(rgbValues, numpy.float64)
    
    # Convert the color labels into a one-hot feature array.
    # Text labels for each array position are in the classes_ list on the binarizer.
    labelBinarizer = LabelBinarizer()
    oneHotLabels = labelBinarizer.fit_transform(colorNames)
    numColors = len(labelBinarizer.classes_)
    colorLabels = labelBinarizer.classes_
    
    # Partition the data into training and testing splits using 80% of
    # the data for training and the remaining 20% for testing.
    (trainingColors, testColors, trainingOneHotLabels, testOneHotLabels) = train_test_split(
        rgbNumpyArray, oneHotLabels, test_size=0.2)

    # Hyperparameters to define the network shape.
    numFullyConnectedPerceptrons = numColors * perceptronsPerColorName
    
    model = Sequential([
        # Layer 1: Fully connected layer with ReLU activation.
        Dense(numFullyConnectedPerceptrons, activation='relu', kernel_initializer='TruncatedNormal', input_shape=(3,)),

        # Outputs: SoftMax activation to get probabilities by color.
        Dense(numColors, activation='softmax')
    ])

    print(model.summary())

    # Compile for categorization.
    model.compile(
        optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, decay = 1e-6, nesterov = True),
        loss = 'categorical_crossentropy',
        metrics = [ 'accuracy' ])

    history = model.fit(trainingColors, trainingOneHotLabels, epochs=epochs, batch_size=batchSize)

    print("")
    print("Scoring result against test data after training with training data:")
    score = model.evaluate(testColors, testOneHotLabels, batch_size=batchSize)
    
    print("")
    print("Score: loss=%1.4f, accuracy=%1.4f" % (score[0], score[1]))
    return (model, colorLabels)

def createMoreTrainingData(colorNameToRGBMap):
    # The incoming color map is not typically going to be oversubscribed with e.g.
    # extra 'red' samples pointing to slightly different colors. We generate a
    # training dataset by perturbing each color by a small amount positive and
    # negative. We do this for each color individually, by pairs, and for all three
    # at once, for each positive and negative value, resulting in dataset that is
    # many times as large.
    perturbValues = [ 0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03 ]
    rgbValues = []
    labels = []
    for colorName, rgb in colorNameToRGBMap.items():
        reds = []
        greens = []
        blues = []
        for perturb in perturbValues:
            if rgb[0] + perturb <= 1.0:
                reds.append(rgb[0] + perturb)
            if perturb != 0.0 and rgb[0] - perturb >= 0.0:
                reds.append(rgb[0] - perturb)
            if rgb[1] + perturb <= 1.0:
                greens.append(rgb[1] + perturb)
            if perturb != 0.0 and rgb[1] - perturb >= 0.0:
                greens.append(rgb[1] - perturb)
            if rgb[2] + perturb <= 1.0:
                blues.append(rgb[2] + perturb)
            if perturb != 0.0 and rgb[2] - perturb >= 0.0:
                blues.append(rgb[2] - perturb)
        for red in reds:
            for green in greens:
                for blue in blues:
                    rgbValues.append((red, green, blue))
                    labels.append(colorName)
    return (rgbValues, labels)

Using TensorFlow backend.


And now our expanded list of 96 colors from [Jenny's page](http://www.jennyscrayoncollection.com/2017/10/complete-list-of-current-crayola-crayon.html), and let's go!

In [2]:
def rgbToFloat(r, g, b):  # r, g, b in 0-255 range
    return (float(r) / 255.0, float(g) / 255.0, float(b) / 255.0)

# http://www.jennyscrayoncollection.com/2017/10/complete-list-of-current-crayola-crayon.html
colorMap = {
    # 8-crayon box colors
    'red': rgbToFloat(238, 32, 77),
    'yellow': rgbToFloat(252, 232, 131),
    'blue': rgbToFloat(31, 117, 254),
    'brown': rgbToFloat(180, 103, 77),
    'orange': rgbToFloat(255, 117, 56),
    'green': rgbToFloat(28, 172, 20),
    'violet': rgbToFloat(146, 110, 174),
    'black': rgbToFloat(35, 35, 35),

    # Additional for 16-count box
    'red-violet': rgbToFloat(192, 68, 143),
    'red-orange': rgbToFloat(255, 117, 56),
    'yellow-green': rgbToFloat(197, 227, 132),
    'blue-violet': rgbToFloat(115, 102, 189),
    'carnation-pink': rgbToFloat(255, 170, 204),
    'yellow-orange': rgbToFloat(255, 182, 83),
    'blue-green': rgbToFloat(25, 158, 189),
    'white': rgbToFloat(237, 237, 237),

    # Additional for 24-count box
    'violet-red': rgbToFloat(247, 83 ,148),
    'apricot': rgbToFloat(253, 217, 181),
    'cerulean': rgbToFloat(29, 172, 214),
    'indigo': rgbToFloat(93, 118, 203),
    'scarlet': rgbToFloat(242, 40, 71),
    'green-yellow': rgbToFloat(240, 232, 145),
    'bluetiful': rgbToFloat(46, 80, 144),
    'gray': rgbToFloat(149, 145, 140),
    
    # Additional for 32-count box
    'chestnut': rgbToFloat(188, 93, 88),
    'peach': rgbToFloat(255, 207, 171),
    'sky-blue': rgbToFloat(128, 215, 235),
    'cadet-blue': rgbToFloat(176, 183, 198),
    'melon': rgbToFloat(253, 188, 180),
    'tan': rgbToFloat(250, 167, 108),
    'wisteria': rgbToFloat(205, 164, 222),
    'timberwolf': rgbToFloat(219, 215, 210),

    # Additional for 48-count box
    'lavender': rgbToFloat(252, 180, 213),
    'burnt-sienna': rgbToFloat(234, 126, 93),
    'olive-green': rgbToFloat(186, 184, 108),
    'purple-mountains-majesty': rgbToFloat(157, 129, 186),
    'salmon': rgbToFloat(255, 155, 170),
    'macaroni-and-cheese': rgbToFloat(255, 189, 136),
    'granny-smith-apple': rgbToFloat(168, 228, 160),
    'sepia': rgbToFloat(165, 105, 79),
    'mauvelous': rgbToFloat(239, 152, 170),
    'goldenrod': rgbToFloat(255, 217, 117),
    'sea-green': rgbToFloat(159, 226, 191),
    'raw-sienna': rgbToFloat(214, 138, 89),
    'mahogany': rgbToFloat(205, 74, 74),
    'spring-green': rgbToFloat(236, 234, 190),
    'cornflower': rgbToFloat(154, 206, 235),
    'tumbleweed': rgbToFloat(222, 170, 136),
    
    # Additional for 64-count box
    'magenta': rgbToFloat(246, 100, 175),
    'bittersweet': rgbToFloat(253, 124, 110),
    'forest-green': rgbToFloat(109, 174, 129),
    'periwinkle': rgbToFloat(197, 208, 230),
    'wild-strawberry': rgbToFloat(255, 67, 164),
    'burnt-orange': rgbToFloat(255, 127, 73),
    'robin-egg-blue': rgbToFloat(31, 206, 203),
    'orchid': rgbToFloat(230, 168, 215),
    'tickle-me-pink': rgbToFloat(252, 137, 172),
    'gold': rgbToFloat(231, 198, 151),
    'turquoise-blue': rgbToFloat(119, 221, 231),
    'plum': rgbToFloat(142, 69, 133),
    'brick-red': rgbToFloat(203, 65, 84),
    'asparagus': rgbToFloat(135, 169, 107),
    'pacific-blue': rgbToFloat(28, 169, 201),
    'silver': rgbToFloat(205, 197, 194),

    # Additional for 96-count box
    'fuchsia': rgbToFloat(195, 100, 197),
    'maroon': rgbToFloat(200, 56, 90),
    'neon-carrot': rgbToFloat(255, 163, 67),
    'jungle-green': rgbToFloat(59, 176, 143),
    'shocking-pink': rgbToFloat(251, 126, 253),
    'radical-red': rgbToFloat(255, 73, 107),
    'sunglow': rgbToFloat(255, 207, 72),
    'tropical-rain-forest': rgbToFloat(23, 128, 109),
    'purple-pizzazz': rgbToFloat(255, 29, 206),
    'wild-watermelon': rgbToFloat(252, 108, 133),
    'laser-lemon': rgbToFloat(253, 252, 116),
    'pine-green': rgbToFloat(21, 128, 120),
    'hot-magenta': rgbToFloat(255, 29, 206),
    'vivid-tangerine': rgbToFloat(255, 160, 137),
    'unmellow-yellow': rgbToFloat(253, 252, 116),
    'navy-blue': rgbToFloat(25, 116, 210),
    'razzle-dazzle-rose': rgbToFloat(255, 72, 208),
    'copper': rgbToFloat(221, 148, 117),
    'inchworm': rgbToFloat(178, 236, 93),
    'denim': rgbToFloat(43, 108, 196),
    'cerise': rgbToFloat(221, 68, 146),
    'atomic-tangerine': rgbToFloat(255, 164, 116),
    'screamin-green': rgbToFloat(118, 255, 122),
    'midnight-blue': rgbToFloat(26, 72, 118),
    'jazzberry-jam': rgbToFloat(202, 55, 103),
    'outrageous-orange': rgbToFloat(255, 110, 74),
    'electric-lime': rgbToFloat(29, 249, 20),
    'wild-blue-wonder': rgbToFloat(162, 173, 208),
    'razzmatazz': rgbToFloat(227, 37, 107),
    'mango-tango': rgbToFloat(255, 130, 67),
    'shamrock': rgbToFloat(69, 206, 162),
    'royal-purple': rgbToFloat(120, 81, 169),
}

(rgbValues, colorNames) = createMoreTrainingData(colorMap)
(colorModel, colorLabels) = train(rgbValues, colorNames)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 384)               1536      
_________________________________________________________________
dense_2 (Dense)              (None, 96)                36960     
Total params: 38,496
Trainable params: 38,496
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
143072/143072 [==============================] - 19s 129us/step - loss: 1.7882 - acc: 0.5574
Epoch 2/3
143072/143072 [==============================] - 18s 122us/step - loss: 0.5246 - acc: 0.8573
Epoch 3/3
143072/143072 [==============================] - 17s 121us/step - loss: 0.3532 - acc: 0.8973

Scoring result against test data after training with training data:
35769/35769 [==================

On my machine I got a scoring result from test data that had lower loss than the training steps. Not bad. Also note that as our colors go up, the number of perturbed data values goes up, 140K for the training data and 35K for the test data. Good thing we increased the batch size to 16!

0.3 loss is not that great, but you can play with the epochs and batch size and perceptron count below to see how much better you can do.

Try out the network against the color slider.

In [3]:
from ipywidgets import interact
from IPython.core.display import display, HTML

def displayColor(r, g, b):
    rInt = min(255, max(0, int(r * 255.0)))
    gInt = min(255, max(0, int(g * 255.0)))
    bInt = min(255, max(0, int(b * 255.0)))
    hexColor = "#%02X%02X%02X" % (rInt, gInt, bInt)
    display(HTML('<div style="width: 50%; height: 50px; background: ' + hexColor + ';"></div>'))

numPredictionsToShow = 5
@interact(r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))
def getTopPredictionsFromModel(r, g, b):
    testColor = numpy.array([ (r, g, b) ])
    predictions = colorModel.predict(testColor, verbose=0)  # Predictions shape (1, numColors)
    predictions *= 100.0
    predColorTuples = []
    for i in range(0, len(colorLabels)):
        predColorTuples.append((predictions[0][i], colorLabels[i]))
    predAndNames = numpy.array(predColorTuples, dtype=[('pred', float), ('colorName', 'U50')])
    sorted = numpy.sort(predAndNames, order=['pred', 'colorName'])
    sorted = sorted[::-1]  # reverse rows to get highest on top
    for i in range(0, numPredictionsToShow):
        print("%2.1f" % sorted[i][0] + "%", sorted[i][1])
    displayColor(r, g, b)

interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

Even with only 3 epochs I'm finding the predictions rather good. See if you agree.

Here's the hyperparameter slider version so you can try out different epochs, batch sizes, and perceptrons:

In [4]:
@interact(epochs = (1, 10), perceptronsPerColorName = (1, 12), batchSize = (1, 50))
def trainModel(epochs=5, perceptronsPerColorName=5, batchSize=8):
    global colorModel
    global colorLabels
    (colorModel, colorLabels) = train(rgbValues, colorNames, epochs=epochs, perceptronsPerColorName=perceptronsPerColorName, batchSize=batchSize)

interactive(children=(IntSlider(value=5, description='epochs', max=10, min=1), IntSlider(value=5, description=…

In [5]:
interact(getTopPredictionsFromModel, r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))

interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

<function __main__.getTopPredictionsFromModel(r, g, b)>

### Coming up...
We'll try to get an understanding for the idea of *backpropagation*, which is how the network training works to increase the accuracy and reduce the loss, at least if the network can learn properly.